# Find the Repos Available in your Database, and What Repository Groups They Are In

## Connect to your database

In [3]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Available Respositories

In [4]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived 
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived
0,chaoss,25155,augur,25440,Parent not available,0
1,chaoss,25155,augur-spdx,25460,Parent not available,0
2,chaoss,25155,governance,25449,Parent not available,0
3,chaoss,25155,grimoirelab,25448,Parent not available,0
4,chaoss,25155,grimoirelab-bestiary,25443,Parent not available,0
...,...,...,...,...,...,...
73,SaltStack,60017,salt-winrepo-ng,30356,Parent not available,0
74,SaltStack,60017,salty-vagrant,30362,Parent not available,0
75,SaltStack,60017,sublime-text,30347,Parent not available,0
76,SaltStack,60017,takara,30380,Parent not available,1


rg_name          object
repo_group_id     int64
repo_name        object
repo_id           int64
forked_from      object
repo_archived     int64
dtype: object

### Create a Simpler List for quickly Identifying repo_group_id's and repo_id's for other queries

In [3]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT b.repo_id,
                a.repo_group_id,
                b.repo_name,
                a.rg_name
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,repo_id,repo_group_id,repo_name,rg_name
0,27192,25177,oss-httpd-build,appsuite
1,27188,25177,springtrader,appsuite
2,27185,25177,sqlf-cloud,appsuite
3,27184,25177,sqlf-demo,appsuite
4,27190,25177,tcs-embed-example,appsuite
...,...,...,...,...
3675,28036,25182,velero-plugin-for-csi,vmware-tanzu
3676,28041,25182,velero-plugin-for-gcp,vmware-tanzu
3677,28040,25182,velero-plugin-for-microsoft-azure,vmware-tanzu
3678,28046,25182,velero-plugin-for-vsphere,vmware-tanzu


repo_id           int64
repo_group_id     int64
repo_name        object
rg_name          object
dtype: object